<a href="https://colab.research.google.com/github/Divyansh-Bari/Sentiment-Analysis-using-RoBERTa/blob/main/Ukraine_Russia_Twitter_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nest_asyncio twikit

import asyncio
import nest_asyncio
from twikit import Client, TooManyRequests
import time
from datetime import datetime
import csv
from configparser import ConfigParser
from random import randint

# Apply nest_asyncio to avoid RuntimeError in Colab
nest_asyncio.apply()

MINIMUM_TWEETS = 100
QUERY = 'Ukraine-Russia War lang:en until:2025-04-01 since:2025-01-01'


async def get_tweets(client, tweets):
    if tweets is None:
        print(f'{datetime.now()} - Getting tweets...')
        tweets = await client.search_tweet(QUERY, product='Top')
    else:
        wait_time = randint(5, 10)
        print(f'{datetime.now()} - Getting next tweets after {wait_time} seconds ...')
        time.sleep(wait_time)
        tweets = await tweets.next()

    return tweets


async def main():
    config = ConfigParser()
    config.read('config.ini')
    username = config['X']['username']
    email = config['X']['email']
    password = config['X']['password']

    with open('tweets.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Tweet_count', 'Username', 'Text', 'Created At', 'Retweets', 'Likes'])

    client = Client(language='en-US')
    await client.login(auth_info_1=username, auth_info_2=email, password=password)
    client.save_cookies('cookies.json')
    client.load_cookies('cookies.json')

    tweet_count = 0
    tweets = None

    while tweet_count < MINIMUM_TWEETS:
        try:
            tweets = await get_tweets(client, tweets)
        except TooManyRequests as e:
            rate_limit_reset = datetime.fromtimestamp(e.rate_limit_reset)
            print(f'{datetime.now()} - Rate limit reached. Waiting until {rate_limit_reset}')
            wait_time = (rate_limit_reset - datetime.now()).total_seconds()
            time.sleep(wait_time)
            continue

        if not tweets:
            print(f'{datetime.now()} - No more tweets found')
            break

        for tweet in tweets:
            tweet_count += 1
            tweet_data = [tweet_count, tweet.user.name, tweet.text, tweet.created_at, tweet.retweet_count, tweet.favorite_count]

            with open('tweets.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(tweet_data)

        print(f'{datetime.now()} - Got {tweet_count} tweets')

    print(f'{datetime.now()} - Done! Got {tweet_count} tweets found')


# ✅ Run the async function in Colab
loop = asyncio.get_event_loop()
loop.run_until_complete(main())


In order to protect your account from suspicious activity, we've sent a confirmation code to di******************@g****.***. Enter it below to sign in.	
>>> 4gwga9e5
2025-04-05 07:36:19.007289 - Getting tweets...
2025-04-05 07:36:20.443864 - Got 19 tweets
2025-04-05 07:36:20.443957 - Getting next tweets after 5 seconds ...
2025-04-05 07:36:26.798014 - Got 39 tweets
2025-04-05 07:36:26.798107 - Getting next tweets after 8 seconds ...
2025-04-05 07:36:36.326743 - Got 59 tweets
2025-04-05 07:36:36.326840 - Getting next tweets after 10 seconds ...
2025-04-05 07:36:47.668191 - Got 79 tweets
2025-04-05 07:36:47.668288 - Getting next tweets after 8 seconds ...
2025-04-05 07:36:57.155598 - Got 99 tweets
2025-04-05 07:36:57.155701 - Getting next tweets after 6 seconds ...
2025-04-05 07:37:04.274952 - Got 119 tweets
2025-04-05 07:37:04.275304 - Done! Got 119 tweets found
